# Preprocessing and saving data
## This notebook contains all preprocessing of the [Adult dataset](https://archive.ics.uci.edu/ml/datasets/Adult)

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Columns are from here: https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names

In [2]:
names = [
    "age",
    "workclass",
    "fnlwgt",
    "education",
    "education_num",
    "marital_status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "capital_gain",
    "capital_loss",
    "hours_per_week",
    "native_country",
    "salary",
]

In [3]:
def clean_string(s):
    """
    Helper function that strips leading / trailing whitespace, lower
    cases, and replaces hyphens with underscores.
    """
    return s.strip().lower().replace("-", "_")

# Data contains a "native country" feature. Other than USA and Mexico, many of the features have low numbers of observations, so group them into a single category using this function.

def parse_native_country(country):
    """
    Group countries other than United-States and Mexico into single
    "other" category"
    """
    country = clean_string(country)
    if country == "united_states" or country == "mexico":
        return country
    return "other"

Load train set and apply some basic preprocessing. Categorical features are left as strings for now to be one-hot encoded shortly. We drop fnlwgt as it represents census weights that are not relevant to our analysis, and education-num as it duplicates data present in the education feature which we use instead.

In [6]:
train = (
    pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
        header=None,
        na_values=[" ?"],
        names=names,
    )
    .drop(columns=["fnlwgt", "education_num"])
    # drop all rows with missing values
    .dropna()
    .reset_index(drop=True)
    # simple preprocessing on columns
    .assign(
        # clean all string columns
        education=lambda df: df.education.map(clean_string),
        marital_status=lambda df: df.marital_status.map(clean_string),
        occupation=lambda df: df.occupation.map(clean_string),
        race=lambda df: df.race.map(clean_string),
        relationship=lambda df: df.relationship.map(clean_string),
        workclass=lambda df: df.workclass.map(clean_string),
        # clean and aggregate native_country
        native_country=lambda df: df.native_country.map(parse_native_country),
        # encode binary features as integers
        salary=lambda df: (df.salary == " >50K").astype(np.int32),
        sex=lambda df: (df.sex == " Male").astype(np.int32),
    )
)

In [4]:
test = (
    pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test",
        header=None,
        na_values=[" ?"],
        skiprows=1,
        names=names,
    )
    .drop(columns=["fnlwgt", "education_num"])
    # drop all rows with missing values
    .dropna()
    .reset_index(drop=True)
    # simple preprocessing on columns
    .assign(
        # clean all string columns
        education=lambda df: df.education.map(clean_string),
        marital_status=lambda df: df.marital_status.map(clean_string),
        occupation=lambda df: df.occupation.map(clean_string),
        race=lambda df: df.race.map(clean_string),
        relationship=lambda df: df.relationship.map(clean_string),
        workclass=lambda df: df.workclass.map(clean_string),
        # clean and aggregate native_country
        native_country=lambda df: df.native_country.map(parse_native_country),
        # encode binary features as integers
        # note extra '.' in test set not present in train set
        salary=lambda df: (df.salary == " >50K.").astype(np.int32),
        sex=lambda df: (df.sex == " Male").astype(np.int32),
    )
)

In [8]:
# check data is correct
assert set(train.education) == set(test.education)
assert set(train.race) == set(test.race)
assert set(train.relationship) == set(test.relationship)
assert set(train.marital_status) == set(test.marital_status)

In [9]:
test

,age,workclass,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,salary
0,25,private,11th,never_married,machine_op_inspct,own_child,black,1,0,0,40,united_states,0
1,38,private,hs_grad,married_civ_spouse,farming_fishing,husband,white,1,0,0,50,united_states,0
2,28,local_gov,assoc_acdm,married_civ_spouse,protective_serv,husband,white,1,0,0,40,united_states,1
3,44,private,some_college,married_civ_spouse,machine_op_inspct,husband,black,1,7688,0,40,united_states,1
4,34,private,10th,never_married,other_service,not_in_family,white,1,0,0,30,united_states,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15055,33,private,bachelors,never_married,prof_specialty,own_child,white,1,0,0,40,united_states,0
15056,39,private,bachelors,divorced,prof_specialty,not_in_family,white,0,0,0,36,united_states,0
15057,38,private,bachelors,married_civ_spouse,prof_specialty,husband,white,1,0,0,50,united_states,0
15058,44,private,bachelors,divorced,adm_clerical,own_child,asian_pac_islander,1,5455,0,40,united_states,0


In [10]:
train

,age,workclass,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,salary
0,39,state_gov,bachelors,never_married,adm_clerical,not_in_family,white,1,2174,0,40,united_states,0
1,50,self_emp_not_inc,bachelors,married_civ_spouse,exec_managerial,husband,white,1,0,0,13,united_states,0
2,38,private,hs_grad,divorced,handlers_cleaners,not_in_family,white,1,0,0,40,united_states,0
3,53,private,11th,married_civ_spouse,handlers_cleaners,husband,black,1,0,0,40,united_states,0
4,28,private,bachelors,married_civ_spouse,prof_specialty,wife,black,0,0,0,40,other,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30157,27,private,assoc_acdm,married_civ_spouse,tech_support,wife,white,0,0,0,38,united_states,0
30158,40,private,hs_grad,married_civ_spouse,machine_op_inspct,husband,white,1,0,0,40,united_states,1
30159,58,private,hs_grad,widowed,adm_clerical,unmarried,white,0,0,0,40,united_states,0
30160,22,private,hs_grad,never_married,adm_clerical,own_child,white,1,0,0,20,united_states,0


In [11]:
one_hot_features = [
    "workclass",
    "education",
    "occupation",
    "race",
    "relationship",
    "marital_status",
    "native_country",
]

cts_features = ["age", "capital_gain", "capital_loss", "hours_per_week"]

binary_features = ["sex", "salary"]

train["race"].value_counts()

race
white                 25933
black                  2817
asian_pac_islander      895
amer_indian_eskimo      286
other                   231
Name: count, dtype: int64

In [13]:
train_df = pd.concat(
    [train, pd.get_dummies(train.loc[:, one_hot_features], dtype=np.int32)],
    axis=1,
)

test_df = pd.concat(
    [test, pd.get_dummies(test.loc[:, one_hot_features], dtype=np.int32)],
    axis=1,
)

In [16]:
assert train_df.columns.tolist() == test_df.columns.tolist()

In [ ]:
# split in train/val
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [20]:
artifacts_dir = Path("artifacts")
data_dir = artifacts_dir / "data" / "adult"
original_features = cts_features + one_hot_features + binary_features
train_df[original_features].to_csv(
    data_dir / "processed" / "train.csv", index=False
)
val_df[original_features].to_csv(
    data_dir / "processed" / "val.csv", index=False
)
test_df[original_features].to_csv(
    data_dir / "processed" / "test.csv", index=False
)

In [21]:
ss = StandardScaler()

train_df[cts_features] = ss.fit_transform(train_df[cts_features])
val_df[cts_features] = ss.transform(val_df[cts_features])
test_df[cts_features] = ss.transform(test_df[cts_features])

In [22]:
train_df.drop(columns=one_hot_features).to_csv(
    data_dir / "processed" / "train-one-hot.csv", index=False
)
val_df.drop(columns=one_hot_features).to_csv(
    data_dir / "processed" / "val-one-hot.csv", index=False
)
test_df.drop(columns=one_hot_features).to_csv(
    data_dir / "processed" / "test-one-hot.csv", index=False
)

In [23]:
train_df

,age,workclass,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,...,marital_status_divorced,marital_status_married_af_spouse,marital_status_married_civ_spouse,marital_status_married_spouse_absent,marital_status_never_married,marital_status_separated,marital_status_widowed,native_country_mexico,native_country_other,native_country_united_states
18406,1.108936,self_emp_not_inc,9th,married_civ_spouse,farming_fishing,husband,white,1,-0.147741,-0.218133,...,0,0,1,0,0,0,0,0,0,1
22563,0.805386,self_emp_not_inc,bachelors,married_civ_spouse,farming_fishing,husband,white,1,-0.147741,-0.218133,...,0,0,1,0,0,0,0,0,0,1
9267,-0.788255,private,some_college,married_civ_spouse,exec_managerial,husband,asian_pac_islander,1,-0.147741,-0.218133,...,0,0,1,0,0,0,0,0,1,0
23824,1.640150,private,10th,widowed,tech_support,unmarried,white,0,-0.147741,-0.218133,...,0,0,0,0,0,0,1,0,0,1
12799,1.108936,private,hs_grad,married_civ_spouse,craft_repair,husband,black,1,-0.147741,-0.218133,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29802,0.122397,private,hs_grad,divorced,transport_moving,not_in_family,white,1,-0.147741,-0.218133,...,1,0,0,0,0,0,0,0,0,1
5390,0.198285,self_emp_not_inc,some_college,married_civ_spouse,machine_op_inspct,husband,white,1,0.263907,-0.218133,...,0,0,1,0,0,0,0,0,0,1
860,-0.105266,private,hs_grad,married_civ_spouse,craft_repair,husband,white,1,-0.147741,-0.218133,...,0,0,1,0,0,0,0,0,0,1
15795,1.336599,self_emp_not_inc,11th,married_civ_spouse,other_service,wife,white,0,-0.147741,-0.218133,...,0,0,1,0,0,0,0,0,0,1
